In [0]:
# Alteração

from pyspark.sql.functions import col, sum, round, date_trunc, year, month, lit, current_timestamp, count, avg

# Definições de Variáveis
CATALOG_NAME = "workspace"
SCHEMA_NAME = "projeto_medalhao_ecommerce"

print(f"Iniciando Setup e agregações para o Schema: {SCHEMA_NAME}")

In [0]:
# Alteração

#  Célula 3: Pipeline Principal (Contém o GOLD 1 e GOLD 2)

# Lemos as tabelas usando o nome do Schema
df_clientes = spark.read.table(f"{SCHEMA_NAME}.silver_clientes")
df_pedidos = spark.read.table(f"{SCHEMA_NAME}.silver_pedidos")
df_itens_pedido = spark.read.table(f"{SCHEMA_NAME}.silver_itens_pedido")
df_produtos = spark.read.table(f"{SCHEMA_NAME}.silver_produtos")


# ==============================================================================
# PASSO 1: CRIAÇÃO DO DATAFRAME BASE COM MÉTRICAS (DF_FATO_CALCULADA)
# ==============================================================================

# 1.1 Combinar Pedidos (Fato) com Itens de Pedido
df_fato_base = df_itens_pedido.alias("itens") \
    .join(df_pedidos.alias("pedidos"), col("itens.id_pedido") == col("pedidos.id_pedido"), "inner") \
    .select(
        col("itens.id_pedido"),
        col("itens.id_produto"),
        col("pedidos.id_cliente"),
        col("pedidos.data_pedido"), 
        col("itens.quantity").alias("quantidade"), 
        col("itens.preco").alias("preco_unitario") 
    )

# 1.2 Calcular Receita e Adicionar Colunas Temporais (ano, mes)
df_fato_calculada = df_fato_base \
    .withColumn("receita_total", round(col("quantidade") * col("preco_unitario"), 2)) \
    .withColumn("ano", year(col("data_pedido"))) \
    .withColumn("mes", month(col("data_pedido"))) \
    .withColumn("data_chave", date_trunc("day", col("data_pedido")))


# ==============================================================================
# PASSO 2: CRIAÇÃO DO GOLD 1 (Fato Detalhada)
# ==============================================================================

df_final_gold = df_fato_calculada.alias("fato") \
    .join(df_produtos.alias("prod"), col("fato.id_produto") == col("prod.id_produto"), "left") \
    .join(df_clientes.alias("cli"), col("fato.id_cliente") == col("cli.id_cliente"), "left") \
    .select(
        # Adicionar o ID do Pedido para que possa ser contado no GOLD 2
        col("fato.id_pedido"), 

        # Métricas
        col("fato.receita_total"), col("fato.quantidade"), col("fato.preco_unitario"),
        
        # Dimensões Tempo
        col("fato.data_chave").alias("data_venda"), col("fato.ano"), col("fato.mes"),

        # Dimensões Produto
        col("prod.id_produto"), col("prod.categoria_produto"),
        
        # Dimensões Cliente
        col("cli.id_cliente"), col("cli.nome_cliente"), col("cli.pais"),
        
        # Metadado
        lit("faturamento").alias("regra_negocio_gold"),
        current_timestamp().alias("processamento_gold_ts")
    )


# ==============================================================================
# PASSO 3: CRIAÇÃO DO GOLD 2 (Agregado para BI)
# ==============================================================================

# CRIAÇÃO do DataFrame df_agregado_mensal (Deve vir antes da escrita)
df_agregado_mensal = df_final_gold \
    .groupBy("ano", "mes", "categoria_produto") \
    .agg(
        sum(col("receita_total")).alias("receita_mensal_total"),
        count(col("id_pedido")).alias("total_pedidos_mes"),
        avg(col("receita_total")).alias("ticket_medio")
    )

# Escrita GOLD 2
df_agregado_mensal.write.format("delta") \
    .option("overwriteSchema", "true") \
    .mode("overwrite") \
    .saveAsTable(f"{SCHEMA_NAME}.gold_sumario_mensal_bi")
print("-> Tabela GOLD: gold_sumario_mensal_bi (Agregado para BI) criada/atualizada com sucesso.")